# Medium Analyzer - Advanced Examples

This notebook contains comprehensive examples of real-world media formulations and advanced analysis techniques.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ftwigg/biochemE_media_analysis/blob/main/notebooks/examples.ipynb)

## Setup

In [ ]:
# Clone repository and install requirements
!git clone https://github.com/ftwigg/biochemE_media_analysis.git 2>/dev/null || echo "Repo already cloned"
%cd biochemE_media_analysis
!pip install -r requirements.txt -q

# Additional packages for visualization
!pip install matplotlib seaborn -q

print("✅ Setup complete!")

In [ ]:
# Import all necessary modules
from medium_analyzer import ComponentLibrary, Component, ComplexComponent, Recipe, MediumAnalysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

# Set up plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Initialize component library
lib = ComponentLibrary()

## 1. Real-World Bacterial Media

### LB (Luria-Bertani) Medium

In [ ]:
# First, add tryptone to our library
tryptone = ComplexComponent(
    name="Tryptone",
    formula="Complex",
    elemental_composition={
        'C': 42.0, 'H': 7.0, 'N': 13.5, 'O': 28.0, 
        'P': 2.5, 'S': 0.6
    },
    mw=100,
    metadata={'type': 'pancreatic digest of casein'}
)
lib.add_component(tryptone)

# Create LB medium
lb_medium = Recipe("LB Medium", "Luria-Bertani medium for E. coli")
lb_medium.add_component(tryptone, 10.0, "g/L")
lb_medium.add_component(lib.get_component("Yeast Extract"), 5.0, "g/L")
lb_medium.add_component(lib.get_component("Sodium Chloride"), 10.0, "g/L")

lb_analysis = lb_medium.analyze()
lb_analysis.print_report()

### TB (Terrific Broth) Medium

In [ ]:
# TB is richer than LB and has phosphate buffer
tb_medium = Recipe("TB Medium", "Terrific Broth for high-density E. coli culture")
tb_medium.add_component(tryptone, 12.0, "g/L")
tb_medium.add_component(lib.get_component("Yeast Extract"), 24.0, "g/L")
tb_medium.add_component(lib.get_component("Glycerol"), 4.0, "g/L")
tb_medium.add_component(lib.get_component("Potassium Phosphate Monobasic"), 2.31, "g/L")
tb_medium.add_component(lib.get_component("Potassium Phosphate Dibasic"), 12.54, "g/L")

tb_analysis = tb_medium.analyze()
print("Terrific Broth Analysis:")
print(f"C/N Ratio: {tb_analysis.get_cn_ratio():.2f}")
print(f"Total Mass: {sum(c for _, c in tb_analysis.composition):.1f} g/L")

### M9 Minimal Medium with Supplements

In [ ]:
# Add trace elements
trace_elements = [
    Component("Iron(II) Sulfate Heptahydrate", "FeSO4·7H2O"),
    Component("Manganese(II) Chloride Tetrahydrate", "MnCl2·4H2O"),
    Component("Zinc Sulfate Heptahydrate", "ZnSO4·7H2O"),
    Component("Cobalt(II) Chloride Hexahydrate", "CoCl2·6H2O"),
    Component("Copper(II) Sulfate Pentahydrate", "CuSO4·5H2O"),
    Component("Sodium Molybdate Dihydrate", "Na2MoO4·2H2O"),
    Component("Boric Acid", "H3BO3")
]

for comp in trace_elements:
    lib.add_component(comp)

# Create M9 with trace elements
m9_complete = Recipe("M9 Complete", "M9 minimal medium with trace elements")

# Base M9 components
m9_complete.add_component(lib.get_component("Glucose"), 4.0, "g/L")
m9_complete.add_component(lib.get_component("Ammonium Sulfate"), 1.0, "g/L")
m9_complete.add_component(lib.get_component("Potassium Phosphate Monobasic"), 3.0, "g/L")
m9_complete.add_component(lib.get_component("Sodium Chloride"), 0.5, "g/L")
m9_complete.add_component(lib.get_component("Magnesium Sulfate Heptahydrate"), 0.24, "g/L")
m9_complete.add_component(lib.get_component("Calcium Chloride Dihydrate"), 0.011, "g/L")

# Trace elements (typical concentrations)
m9_complete.add_component(lib.get_component("Iron(II) Sulfate Heptahydrate"), 0.001, "g/L")
m9_complete.add_component(lib.get_component("Manganese(II) Chloride Tetrahydrate"), 0.0001, "g/L")
m9_complete.add_component(lib.get_component("Zinc Sulfate Heptahydrate"), 0.0001, "g/L")

m9_analysis = m9_complete.analyze()
print("M9 Complete - Trace Metals (ppm):")
for metal, conc in m9_analysis.get_trace_metals().items():
    if conc > 0:
        print(f"  {metal}: {conc:.4f} ppm")

## 2. Yeast Media

### YPD (Yeast Extract Peptone Dextrose)

In [ ]:
# Add peptone (different from tryptone)
peptone = ComplexComponent(
    name="Peptone",
    formula="Complex",
    elemental_composition={
        'C': 40.0, 'H': 7.5, 'N': 15.0, 'O': 27.0,
        'P': 2.0, 'S': 0.7
    },
    mw=100,
    metadata={'type': 'animal peptone'}
)
lib.add_component(peptone)

# YPD Medium
ypd_medium = Recipe("YPD Medium", "Standard yeast growth medium")
ypd_medium.add_component(lib.get_component("Yeast Extract"), 10.0, "g/L")
ypd_medium.add_component(peptone, 20.0, "g/L")
ypd_medium.add_component(lib.get_component("Glucose"), 20.0, "g/L")  # 2% glucose

ypd_analysis = ypd_medium.analyze()
ypd_analysis.print_report()

### Synthetic Complete (SC) Medium

In [ ]:
# Add amino acids
amino_acids = [
    Component("L-Alanine", "C3H7NO2"),
    Component("L-Arginine", "C6H14N4O2"),
    Component("L-Asparagine", "C4H8N2O3"),
    Component("L-Aspartic Acid", "C4H7NO4"),
    Component("L-Cysteine", "C3H7NO2S"),
    Component("L-Glutamic Acid", "C5H9NO4"),
    Component("L-Glycine", "C2H5NO2"),
    Component("L-Histidine", "C6H9N3O2"),
    Component("L-Isoleucine", "C6H13NO2"),
    Component("L-Leucine", "C6H13NO2"),
    Component("L-Lysine", "C6H14N2O2"),
    Component("L-Methionine", "C5H11NO2S"),
    Component("L-Phenylalanine", "C9H11NO2"),
    Component("L-Proline", "C5H9NO2"),
    Component("L-Serine", "C3H7NO3"),
    Component("L-Threonine", "C4H9NO3"),
    Component("L-Tryptophan", "C11H12N2O2"),
    Component("L-Tyrosine", "C9H11NO3"),
    Component("L-Valine", "C5H11NO2")
]

for aa in amino_acids:
    lib.add_component(aa)

# Create SC medium (simplified version)
sc_medium = Recipe("SC Medium", "Synthetic complete medium for yeast")

# Carbon source
sc_medium.add_component(lib.get_component("Glucose"), 20.0, "g/L")

# Nitrogen source
sc_medium.add_component(lib.get_component("Ammonium Sulfate"), 5.0, "g/L")

# Add amino acids (typical concentrations)
amino_acid_conc = {
    "L-Leucine": 0.380, "L-Lysine": 0.030, "L-Methionine": 0.020,
    "L-Tryptophan": 0.020, "L-Histidine": 0.020, "L-Phenylalanine": 0.050,
    "L-Arginine": 0.020, "L-Tyrosine": 0.030, "L-Valine": 0.150,
    "L-Threonine": 0.200, "L-Isoleucine": 0.030, "L-Glutamic Acid": 0.100,
    "L-Aspartic Acid": 0.100, "L-Serine": 0.375
}

for aa_name, conc in amino_acid_conc.items():
    sc_medium.add_component(lib.get_component(aa_name), conc, "g/L")

# Salts
sc_medium.add_component(lib.get_component("Potassium Phosphate Monobasic"), 1.0, "g/L")
sc_medium.add_component(lib.get_component("Magnesium Sulfate Heptahydrate"), 0.5, "g/L")
sc_medium.add_component(lib.get_component("Sodium Chloride"), 0.1, "g/L")
sc_medium.add_component(lib.get_component("Calcium Chloride Dihydrate"), 0.1, "g/L")

sc_analysis = sc_medium.analyze()
print(f"SC Medium - Total components: {len(sc_medium.components)}")
print(f"C/N Ratio: {sc_analysis.get_cn_ratio():.2f}")
print(f"Total amino acids: {sum(amino_acid_conc.values()):.3f} g/L")

## 3. Cell Culture Media (Mammalian)

### Simplified DMEM (Dulbecco's Modified Eagle Medium)

In [ ]:
# Add vitamins
vitamins = [
    Component("Thiamine HCl", "C12H17ClN4OS·HCl"),
    Component("Riboflavin", "C17H20N4O6"),
    Component("Niacin", "C6H5NO2"),
    Component("Pantothenic Acid", "C9H17NO5"),
    Component("Pyridoxine HCl", "C8H11NO3·HCl"),
    Component("Biotin", "C10H16N2O3S"),
    Component("Folic Acid", "C19H19N7O6"),
    Component("Vitamin B12", "C63H88CoN14O14P"),
    Component("Choline Chloride", "C5H14ClNO"),
    Component("Inositol", "C6H12O6")
]

for vit in vitamins:
    lib.add_component(vit)

# Create DMEM (simplified version)
dmem = Recipe("DMEM", "Dulbecco's Modified Eagle Medium")

# Major components
dmem.add_component(lib.get_component("Glucose"), 4.5, "g/L")  # High glucose
dmem.add_component(lib.get_component("L-Glutamine"), 0.584, "g/L")
dmem.add_component(lib.get_component("Sodium Chloride"), 6.4, "g/L")
dmem.add_component(lib.get_component("Potassium Phosphate Monobasic"), 0.109, "g/L")

# Add essential amino acids
essential_aa = {
    "L-Arginine": 0.084, "L-Cysteine": 0.048, "L-Histidine": 0.042,
    "L-Isoleucine": 0.105, "L-Leucine": 0.105, "L-Lysine": 0.146,
    "L-Methionine": 0.030, "L-Phenylalanine": 0.066, "L-Threonine": 0.095,
    "L-Tryptophan": 0.016, "L-Tyrosine": 0.072, "L-Valine": 0.094
}

for aa, conc in essential_aa.items():
    dmem.add_component(lib.get_component(aa), conc, "g/L")

# Add vitamins (mg/L scale)
dmem.add_component(lib.get_component("Thiamine HCl"), 4.0, "mg/L")
dmem.add_component(lib.get_component("Riboflavin"), 0.4, "mg/L")
dmem.add_component(lib.get_component("Niacin"), 4.0, "mg/L")
dmem.add_component(lib.get_component("Pantothenic Acid"), 4.0, "mg/L")
dmem.add_component(lib.get_component("Biotin"), 0.024, "mg/L")
dmem.add_component(lib.get_component("Folic Acid"), 4.0, "mg/L")
dmem.add_component(lib.get_component("Choline Chloride"), 4.0, "mg/L")
dmem.add_component(lib.get_component("Inositol"), 7.2, "mg/L")

dmem_analysis = dmem.analyze()
print(f"DMEM - Total components: {len(dmem.components)}")
print(f"C/N Ratio: {dmem_analysis.get_cn_ratio():.2f}")
print(f"Total Mass: {sum(c for _, c in dmem_analysis.composition):.2f} g/L")

## 4. Fed-Batch Feeding Strategies

In [ ]:
# Create base batch medium
batch_medium = Recipe("Batch Medium", "Initial batch medium for fermentation")
batch_medium.add_component(lib.get_component("Glucose"), 20.0, "g/L")
batch_medium.add_component(lib.get_component("Ammonium Sulfate"), 5.0, "g/L")
batch_medium.add_component(lib.get_component("Yeast Extract"), 1.0, "g/L")
batch_medium.add_component(lib.get_component("Potassium Phosphate Monobasic"), 3.0, "g/L")
batch_medium.add_component(lib.get_component("Magnesium Sulfate Heptahydrate"), 0.5, "g/L")

# Create concentrated feed solution
feed_solution = Recipe("Feed Solution", "Concentrated feed for fed-batch")
feed_solution.add_component(lib.get_component("Glucose"), 500.0, "g/L")  # 50% glucose
feed_solution.add_component(lib.get_component("Ammonium Sulfate"), 40.0, "g/L")
feed_solution.add_component(lib.get_component("Magnesium Sulfate Heptahydrate"), 10.0, "g/L")

# Simulate fed-batch at different time points
timepoints = [0, 12, 24, 36, 48]  # hours
feed_rates = [0, 2, 4, 6, 8]  # mL/h
reactor_volume = 1000  # mL initial

print("Fed-Batch Simulation:")
print("-" * 60)

for t, feed_rate in zip(timepoints, feed_rates):
    # Calculate total feed added
    total_feed = feed_rate * t  # mL
    dilution = total_feed / reactor_volume
    
    # Create medium at this timepoint
    current_medium = Recipe(f"Medium at {t}h", f"Fed-batch medium at {t} hours")
    current_medium.add_base_recipe(batch_medium, reactor_volume / (reactor_volume + total_feed))
    
    if total_feed > 0:
        current_medium.add_base_recipe(feed_solution, total_feed / (reactor_volume + total_feed))
    
    analysis = current_medium.analyze()
    glucose_conc = next((c for comp, c in analysis.composition if "Glucose" in comp.name), 0)
    cn_ratio = analysis.get_cn_ratio()
    
    print(f"Time: {t:3}h | Feed: {total_feed:4.0f}mL | Glucose: {glucose_conc:6.1f}g/L | C/N: {cn_ratio:5.2f}")

## 5. Medium Comparison and Visualization

In [ ]:
# Compare multiple media
media_to_compare = [
    ("LB", lb_medium),
    ("TB", tb_medium),
    ("M9", m9_complete),
    ("YPD", ypd_medium),
    ("SC", sc_medium),
    ("DMEM", dmem)
]

comparison_data = []
for name, medium in media_to_compare:
    analysis = medium.analyze()
    comparison_data.append({
        'Medium': name,
        'C/N Ratio': analysis.get_cn_ratio() or 0,
        'Total Mass (g/L)': sum(c for _, c in analysis.composition),
        'Carbon (g/L)': analysis._elemental_composition.get('C', 0),
        'Nitrogen (g/L)': analysis._elemental_composition.get('N', 0),
        'Components': len(analysis.composition)
    })

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.sort_values('C/N Ratio')

# Display comparison table
print("\nMedia Comparison Table:")
print(df_comparison.to_string(index=False))

In [ ]:
# Visualize C/N ratios
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# C/N Ratio bar chart
axes[0].bar(df_comparison['Medium'], df_comparison['C/N Ratio'], color='steelblue')
axes[0].set_xlabel('Medium')
axes[0].set_ylabel('C/N Ratio (mass basis)')
axes[0].set_title('C/N Ratios Across Media')
axes[0].tick_params(axis='x', rotation=45)

# Carbon vs Nitrogen scatter
axes[1].scatter(df_comparison['Carbon (g/L)'], df_comparison['Nitrogen (g/L)'], 
                s=100, alpha=0.7, c=range(len(df_comparison)), cmap='viridis')
for i, txt in enumerate(df_comparison['Medium']):
    axes[1].annotate(txt, (df_comparison['Carbon (g/L)'].iloc[i], 
                           df_comparison['Nitrogen (g/L)'].iloc[i]))
axes[1].set_xlabel('Carbon (g/L)')
axes[1].set_ylabel('Nitrogen (g/L)')
axes[1].set_title('Carbon vs Nitrogen Content')
axes[1].grid(True, alpha=0.3)

# Total mass pie chart
axes[2].pie(df_comparison['Total Mass (g/L)'], labels=df_comparison['Medium'], 
            autopct='%1.1f%%', startangle=90)
axes[2].set_title('Relative Total Mass Distribution')

plt.tight_layout()
plt.show()

## 6. Elemental Composition Heatmap

In [ ]:
# Create elemental composition matrix
elements_of_interest = ['C', 'H', 'N', 'O', 'P', 'S', 'K', 'Na', 'Mg', 'Ca']
heatmap_data = []

for name, medium in media_to_compare:
    analysis = medium.analyze()
    elem_comp = analysis.get_elemental_composition("g/L")
    row_data = [elem_comp.get(e, 0) for e in elements_of_interest]
    heatmap_data.append(row_data)

# Create DataFrame
df_heatmap = pd.DataFrame(heatmap_data, 
                          columns=elements_of_interest,
                          index=[name for name, _ in media_to_compare])

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df_heatmap, annot=True, fmt='.3f', cmap='YlOrRd', 
            cbar_kws={'label': 'Concentration (g/L)'})
plt.title('Elemental Composition Across Different Media')
plt.ylabel('Medium')
plt.xlabel('Element')
plt.tight_layout()
plt.show()

## 7. Cost Analysis

In [ ]:
# Add cost information to components ($/kg)
component_costs = {
    "Glucose": 0.5,
    "Ammonium Sulfate": 0.3,
    "Yeast Extract": 25.0,
    "Tryptone": 50.0,
    "Peptone": 45.0,
    "Sodium Chloride": 0.2,
    "Potassium Phosphate Monobasic": 1.5,
    "Potassium Phosphate Dibasic": 1.6,
    "Magnesium Sulfate Heptahydrate": 0.4,
    "Calcium Chloride Dihydrate": 0.6,
    "Glycerol": 1.2,
    "L-Glutamine": 150.0,
}

def calculate_medium_cost(recipe: Recipe, costs: dict) -> tuple:
    """Calculate cost per liter and cost breakdown"""
    total_cost = 0
    cost_breakdown = {}
    
    for comp, conc in recipe.get_total_composition():
        if comp.name in costs:
            cost_per_L = (conc / 1000) * costs[comp.name]  # Convert g/L to kg/L
            total_cost += cost_per_L
            cost_breakdown[comp.name] = cost_per_L
    
    return total_cost, cost_breakdown

# Calculate costs for each medium
print("Medium Cost Analysis:")
print("=" * 60)

cost_results = []
for name, medium in media_to_compare[:4]:  # Just first 4 media
    cost, breakdown = calculate_medium_cost(medium, component_costs)
    cost_results.append({'Medium': name, 'Cost ($/L)': cost})
    
    print(f"\n{name} Medium - Total Cost: ${cost:.2f}/L")
    print("  Top cost contributors:")
    sorted_costs = sorted(breakdown.items(), key=lambda x: x[1], reverse=True)[:3]
    for comp_name, comp_cost in sorted_costs:
        percent = (comp_cost / cost) * 100 if cost > 0 else 0
        print(f"    {comp_name}: ${comp_cost:.3f} ({percent:.1f}%)")

# Cost comparison chart
df_cost = pd.DataFrame(cost_results)
plt.figure(figsize=(8, 5))
plt.bar(df_cost['Medium'], df_cost['Cost ($/L)'], color='green', alpha=0.7)
plt.xlabel('Medium')
plt.ylabel('Cost ($/L)')
plt.title('Medium Cost Comparison')
plt.show()

## 8. Medium Optimization for Target C/N Ratio

In [ ]:
def optimize_cn_ratio(base_recipe: Recipe, target_cn: float, 
                      carbon_source: str, nitrogen_source: str, 
                      lib: ComponentLibrary, tolerance: float = 0.1) -> Recipe:
    """Optimize medium to achieve target C/N ratio"""
    
    optimized = Recipe(f"Optimized {base_recipe.name}", 
                      f"Optimized for C/N = {target_cn}")
    
    # Copy base recipe
    for comp, conc, unit in base_recipe.components:
        optimized.add_component(comp, conc, unit)
    
    # Get current C/N
    current_analysis = optimized.analyze()
    current_cn = current_analysis.get_cn_ratio()
    
    print(f"Starting C/N: {current_cn:.2f}, Target: {target_cn:.2f}")
    
    # Iteratively adjust
    iterations = 0
    max_iterations = 10
    
    while abs(current_cn - target_cn) > tolerance and iterations < max_iterations:
        if current_cn < target_cn:
            # Need more carbon
            optimized.add_component(lib.get_component(carbon_source), 1.0, "g/L")
            print(f"  Added 1 g/L {carbon_source}")
        else:
            # Need more nitrogen
            optimized.add_component(lib.get_component(nitrogen_source), 0.5, "g/L")
            print(f"  Added 0.5 g/L {nitrogen_source}")
        
        current_analysis = optimized.analyze()
        current_cn = current_analysis.get_cn_ratio()
        iterations += 1
        print(f"  Iteration {iterations}: C/N = {current_cn:.2f}")
    
    return optimized

# Example: Optimize M9 medium for C/N = 10
base_m9 = Recipe("Base M9", "Starting M9 medium")
base_m9.add_component(lib.get_component("Glucose"), 4.0, "g/L")
base_m9.add_component(lib.get_component("Ammonium Sulfate"), 1.0, "g/L")
base_m9.add_component(lib.get_component("Potassium Phosphate Monobasic"), 3.0, "g/L")

print("Optimizing M9 Medium for C/N = 10:")
print("-" * 40)

optimized_m9 = optimize_cn_ratio(
    base_m9, 
    target_cn=10.0,
    carbon_source="Glucose",
    nitrogen_source="Ammonium Sulfate",
    lib=lib
)

# Compare before and after
print("\nFinal Comparison:")
print(f"Original C/N: {base_m9.analyze().get_cn_ratio():.2f}")
print(f"Optimized C/N: {optimized_m9.analyze().get_cn_ratio():.2f}")

## 9. Batch Recipe Scaling

In [ ]:
def scale_recipe(recipe: Recipe, from_volume: float, to_volume: float, 
                 name_suffix: str = "") -> Recipe:
    """Scale a recipe from one volume to another"""
    scale_factor = to_volume / from_volume
    
    scaled = Recipe(
        f"{recipe.name} {name_suffix}",
        f"{recipe.description} - Scaled to {to_volume}L"
    )
    
    # Scale all components
    for comp, conc, unit in recipe.components:
        # Note: concentration stays the same, but we'll calculate total amounts
        scaled.add_component(comp, conc, unit)
    
    return scaled, scale_factor

def print_recipe_amounts(recipe: Recipe, volume: float):
    """Print absolute amounts needed for a given volume"""
    print(f"\nRecipe: {recipe.name} - Volume: {volume}L")
    print("=" * 60)
    print(f"{'Component':<35} {'Conc (g/L)':<12} {'Total (g)':<12}")
    print("-" * 60)
    
    total_mass = 0
    for comp, conc in recipe.get_total_composition():
        total_amount = conc * volume
        total_mass += total_amount
        print(f"{comp.name:<35} {conc:<12.3f} {total_amount:<12.3f}")
    
    print("-" * 60)
    print(f"{'Total':<35} {'':<12} {total_mass:<12.3f}")

# Example: Scale LB medium from lab scale to production
volumes = [0.1, 1, 10, 100, 1000]  # Liters

print("LB Medium Scaling from 100mL to 1000L:")
for vol in volumes:
    print_recipe_amounts(lb_medium, vol)

## 10. Export Recipes for Documentation

In [ ]:
def export_recipe_to_markdown(recipe: Recipe, analysis: MediumAnalysis) -> str:
    """Export recipe as markdown documentation"""
    md = f"# {recipe.name}\n\n"
    md += f"**Description:** {recipe.description}\n\n"
    
    md += "## Components\n\n"
    md += "| Component | Concentration | Unit | MW (g/mol) |\n"
    md += "|-----------|--------------|------|------------|\n"
    
    for comp, conc, unit in recipe.components:
        md += f"| {comp.name} | {conc:.3f} | {unit} | {comp.mw:.2f} |\n"
    
    md += "\n## Analysis\n\n"
    md += f"- **C/N Ratio (mass):** {analysis.get_cn_ratio():.2f}\n"
    md += f"- **C/N Ratio (molar):** {analysis.get_molar_cn_ratio():.2f}\n"
    md += f"- **Total Mass:** {sum(c for _, c in analysis.composition):.2f} g/L\n\n"
    
    md += "### Elemental Composition (g/L)\n\n"
    for elem in ['C', 'H', 'N', 'O', 'P', 'S']:
        if elem in analysis._elemental_composition:
            md += f"- **{elem}:** {analysis._elemental_composition[elem]:.4f}\n"
    
    return md

# Export LB medium as markdown
lb_markdown = export_recipe_to_markdown(lb_medium, lb_analysis)
print(lb_markdown)

# Save to file
with open('lb_medium_recipe.md', 'w') as f:
    f.write(lb_markdown)

print("\n✅ Recipe exported to lb_medium_recipe.md")

In [ ]:
# Export all recipes to Excel
def export_all_recipes_to_excel(recipes_list: list, filename: str):
    """Export multiple recipes to Excel with different sheets"""
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        
        # Summary sheet
        summary_data = []
        for name, recipe in recipes_list:
            analysis = recipe.analyze()
            summary_data.append({
                'Recipe': name,
                'Components': len(recipe.components),
                'Total Mass (g/L)': sum(c for _, c in analysis.composition),
                'C/N Ratio': analysis.get_cn_ratio() or 0,
                'C (g/L)': analysis._elemental_composition.get('C', 0),
                'N (g/L)': analysis._elemental_composition.get('N', 0)
            })
        
        df_summary = pd.DataFrame(summary_data)
        df_summary.to_excel(writer, sheet_name='Summary', index=False)
        
        # Individual recipe sheets
        for name, recipe in recipes_list:
            comp_data = []
            for comp, conc in recipe.get_total_composition():
                comp_data.append({
                    'Component': comp.name,
                    'Concentration (g/L)': conc,
                    'MW (g/mol)': comp.mw,
                    'Formula': comp.formula
                })
            
            df_recipe = pd.DataFrame(comp_data)
            # Excel sheet names limited to 31 characters
            sheet_name = name[:31] if len(name) > 31 else name
            df_recipe.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f"✅ Exported {len(recipes_list)} recipes to {filename}")

# Export all media
export_all_recipes_to_excel(media_to_compare, 'media_recipes.xlsx')

# Download link for Colab
try:
    from google.colab import files
    files.download('media_recipes.xlsx')
    files.download('lb_medium_recipe.md')
except ImportError:
    print("Files saved locally (not in Colab)")

## Summary

This notebook demonstrated:

1. **Real-world media formulations** - LB, TB, M9, YPD, SC, DMEM
2. **Complex component handling** - Peptones, yeast extract, amino acids, vitamins
3. **Fed-batch strategies** - Simulating feeding over time
4. **Comparative analysis** - Multiple media comparison
5. **Visualization** - Charts and heatmaps
6. **Cost analysis** - Economic comparison of media
7. **Optimization** - Achieving target C/N ratios
8. **Scaling** - From lab to production scale
9. **Documentation** - Export to Markdown and Excel

### Next Steps:
- Add your own media recipes
- Include more trace elements and vitamins
- Implement pH and osmolality calculations
- Add more complex optimization algorithms
- Create a database of component costs
- Build a web interface for easier access